# Example for running HiggsBounds applying HH limits

In [ ]:
import Higgs.predictions as HP
import Higgs.bounds as HB
import numpy as np
import pandas as pd

pred = HP.Predictions() # create the model predictions
bounds = HB.Bounds('/Path/To/HBDataSet') # load HB dataset

### HiggsPredictions input

add a SM-like Higgs boson with SM-like couplings

In [ ]:
h = pred.addParticle(HP.NeutralScalar("h", "even"))
h.setMass(125.09)
HP.effectiveCouplingInput(h, HP.smLikeEffCouplings)

add second BSM Higgs boson which decays to two $h$ bosons and is produced via gluon fusion

In [ ]:
H = pred.addParticle(HP.NeutralScalar("H", "even"))
H.setDecayWidth("h", "h", 1)
H.setCxn("LHC13", "ggH", 1)

### Run HiggsBounds

setup scanning ranges

In [ ]:
df = pd.DataFrame()

N = 1000

df['mass'] = np.linspace(250, 2001, N)

main function which runs HB and returns list of applied limits involving $H$ bosons

In [ ]:
@np.vectorize
def runHB(mass):
    H.setMass(mass)
    return [a for a in bounds(pred).appliedLimits if "H" in a.contributingParticles()]


derive HB results

In [ ]:
df['appliedLimits'] = runHB(df['mass'])

get list of all applied limits

In [ ]:
limits = list({a.limit() for res in df['appliedLimits'] for a in res})
limits.sort(key=lambda l: str(l.id()))

In [ ]:
@np.vectorize
def get_obsratio(alims, id):
    for a in alims:
        if a.limit().id() == id:
            return a.obsRatio()
    return np.NaN

In [ ]:
for lim in limits:
    df[lim.id()] = get_obsratio(df['appliedLimits'], lim.id())

### Plotting

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator

mpl.rcParams['ytick.right'] = True
mpl.rcParams['xtick.top'] = True
mpl.rcParams['xtick.direction'] = 'in'
mpl.rcParams['ytick.direction'] = 'in'
mpl.rcParams['text.usetex'] = True
mpl.rcParams['font.family'] = 'serif'

def formatProcess(desc):
    parts = desc.split("->")
    finalState = "".join(parts[-2:]).replace(")(X3", "").replace(")", "")
    if len(finalState) > 20:
        return "comb"
    return finalState

In [ ]:
fig, ax = plt.subplots()

for lim in limits:
    ax.plot(
        df['mass'],
        1 / df[lim.id()],
        ls="-" if lim.experiment() == HP.Experiment.ATLAS else "--",
        label="{} {}".format(lim.reference(), formatProcess(lim.processDesc())),
    )

ax.set_yscale("log")

plt.xlim([250, 2000])
plt.ylim([4*10**-3, 2*10**2])

ax.set_xlabel(r"$m_H$ [GeV]")
ax.set_ylabel(r"$\sigma(pp \to H \to h_{125} h_{125})$ [pb]")

ax.xaxis.set_major_locator(MultipleLocator(200))
ax.xaxis.set_minor_locator(MultipleLocator(20))

atlaslims = [
    l for l in ax.get_lines() if l.get_linestyle() == "-" or l.get_linestyle() == ":"
]
cmslims = [
    l for l in ax.get_lines() if l.get_linestyle() == "--" or l.get_linestyle() == "-."
]
legend1 = ax.legend(
    atlaslims,
    [l.get_label() for l in atlaslims],
    loc="upper left",
    title="ATLAS $13\,\mathrm{TeV}$",
    bbox_to_anchor=(1.03, 1.02),
)
legend2 = ax.legend(
    cmslims,
    [l.get_label() for l in cmslims],
    loc="upper left",
    title="CMS $13\,\mathrm{TeV}$",
    bbox_to_anchor=(1.03, 0.502),
)
ax.add_artist(legend1)
ax.add_artist(legend2)

plt.savefig('double.pdf', bbox_inches='tight')
plt.show()